# kTextAugmentation

이 노트북은 kTextAugmentation 패키지를 사용하기 위한 예시입니다.
documentation을 완성하기전까지는 이 노트북을 위주로 설명할 것 같습니다

### 설치하기



In [1]:
%%bash
# mecab 설치 (takes a few minutes)
wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1A3bZzghyP1WvcSVPkf_YID1U5KPXDd5J' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1A3bZzghyP1WvcSVPkf_YID1U5KPXDd5J" -O install_mecab.sh && rm -rf /tmp/cookies.txt
sudo bash ./install_mecab.sh > /dev/null 2>&1

--2021-04-20 08:49:28--  https://docs.google.com/uc?export=download&confirm=d_uz&id=1A3bZzghyP1WvcSVPkf_YID1U5KPXDd5J
Resolving docs.google.com (docs.google.com)... 142.250.97.101, 142.250.97.139, 142.250.97.102, ...
Connecting to docs.google.com (docs.google.com)|142.250.97.101|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0k-28-docs.googleusercontent.com/docs/securesc/k3s41e5j9m84j9sldujhbltjen4r7og4/qp6fjced2k5f57fk6plqj5ruakq31vrj/1618908525000/10000090644219567406/04019969030829579991Z/1A3bZzghyP1WvcSVPkf_YID1U5KPXDd5J?e=download [following]
--2021-04-20 08:49:28--  https://doc-0k-28-docs.googleusercontent.com/docs/securesc/k3s41e5j9m84j9sldujhbltjen4r7og4/qp6fjced2k5f57fk6plqj5ruakq31vrj/1618908525000/10000090644219567406/04019969030829579991Z/1A3bZzghyP1WvcSVPkf_YID1U5KPXDd5J?e=download
Resolving doc-0k-28-docs.googleusercontent.com (doc-0k-28-docs.googleusercontent.com)... 173.194.217.132, 2607:f8b0:400c:c13::84
Connectin

In [1]:
!pip install ktextaug==0.1.9

### 버젼 확인

In [2]:
import ktextaug
print(ktextaug.__version__)

0.1.9


### TextAugmentation() 사용하기
패키지 0.1.9 버젼부턴 기본적으로 TextAugmentation() 을 사용하여 처리하는 것을 권장합니다. multiprocessing 을 이용하여 대용량의 데이터를 빠르게 처리할 수 있도록 만들었습니다. 


In [2]:
from ktextaug import TextAugmentation

sample_text = '달리는 기차 위에 중립은 없다. 미국의 사회 운동가이자 역사학자인 하워드 진이 남긴 격언이다.'
sample_texts = ['프로그램 개발이 끝나고 서비스가 진행된다.', '도움말을 보고 싶다면 --help를 입력하면 된다.']
agent = TextAugmentation(tokenizer="mecab",
                        num_processes=1) # num_process 가 -1 일시 자동으로 가능한 process의 절반으로 할당
print(agent.generate(sample_text))     # default is back_translation
print(agent.generate(sample_texts))

달리는 기차에는 중립이 없습니다. 이것이 미국 사회 운동가이자 역사가 인 하워드 진이 말하는 것입니다.



['프로그램 개발이 완료되면 서비스가 진행됩니다.', '도움말을 보려면 --help를 입력하십시오.']


### 직접 import 해서 사용하기
0.1.8 에서 사용하던 대로 직접 원하는 기능을 import 할 수 있습니다.

현재 불러올 수 있는 함수는 다음과 같습니다. 
"back_translate",
"noise_add",
"random_insert",
"random_delete",
"random_swap",
"synonym_replace",

In [ ]:
from ktextaug import random_swap
from random import Random

text = "이 문장은 변형적 데이터 증강기법의 예시 문장입니다."
tokenizer = bring_it_your_own   # 토크나이저는 어떤 토크나이저를 사용하더라도 상관없습니다.

result = random_swap(text_or_tokens=text,
                     tokenize_fn=tokenizer.tokenize, 
                     rng=Random(2021), # seed
                     n_swaps=2) # random_swap 고유 파라메터. 토큰 시퀀스 내 두 단어의 위치를 변경하는 작업(random swap)을 2회 시행합니다. 
print(result)
# ['이', '문장', '은', '예시', '적', '데이터', '기법', '증강', '의', '문장', '변형', '입니다', '.']

### tokenizer

ktextaug 패키지에서 제공하는 형태소 분석기(토크나이저) 모듈은 다양한 존재하는 토크나이저를 쉽게 사용할 수있도록 만들었습니다. 설치가 필요없는 기본 토크나이저로서 또는 subword tokenizer 을 만들어두었습니다. mecab, komoran 토크나이저의 경우 별도의 설치과정이 필요합니다. 원하는 토크나이저를 사용할 수도 있습니다.
(subword 경우 노이즈와 관련된 vocab을 구축해놓았기 때문에 다른 토크나이저에 비해 noise_add에서 훨씬 강건합니다.)

In [3]:
from ktextaug import TextAugmentation, random_swap
from ktextaug.tokenization_utils import Tokenizer
from random import Random

text ="고래상어는 어류 중 가장 덩치가 큽니다. 하지만 상어라는 이름에 걸맞지 않게 온순해서 사람이 가까이 접근해 헤엄쳐도 경계하지 않고, 이빨 크기도 3mm 안팎이라고 합니다."
# get_tokenize_fn 함수의 사용예시
tokenizer = Tokenizer("mecab")
print(type(tokenizer))
print(tokenizer.tokenize(text))
# tokenize_fn = Tokenizer("komoran")
# print(tokenizer.tokenize(text))
tokenizer = Tokenizer("subword")
print(tokenizer.tokenize(text))

# 이러한 <class 'function'> 또는 <class 'method'> 형태의 함수들은 패키지 내 함수나 TextAugmentation 클래스에 사용됩니다.

agent = TextAugmentation(tokenizer=tokenizer,
#                         tokenize_fn="mecab",
                        num_processes=1)
print(agent.generate(text,
                     mode="random_swap",
                     n_swaps=2))   

print(random_swap(text_or_tokens=text,
                     tokenizer=tokenizer,
                     rng=Random(2021),
                     n_swaps=2))

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


<class 'ktextaug.tokenization_utils.Tokenizer'>
['고래상어', '는', '어류', '중', '가장', '덩치', '가', '큽니다', '.', '하지만', '상어', '라는', '이름', '에', '걸맞', '지', '않', '게', '온순', '해서', '사람', '이', '가까이', '접근', '해', '헤엄쳐', '도', '경계', '하', '지', '않', '고', ',', '이빨', '크기', '도', '3', 'mm', '안팎', '이', '라고', '합니다', '.']
['고래', '##상', '##어는', '어', '##류', '중', '가장', '덩치', '##가', '큽', '##니다', '.', '하지만', '상', '##어', '##라는', '이름에', '걸맞', '##지', '않게', '온', '##순', '##해서', '사람이', '가까이', '접근', '##해', '헤', '##엄', '##쳐', '##도', '경계', '##하지', '않고', ',', '이빨', '크기', '##도', '3', '##mm', '안', '##팎', '##이라고', '합니다', '.']
고래상어는 어류 중 가장 덩치가 큽니다 . 하지만 상어라는 이름에 걸맞도 않게 온순해서 사람이 가까이 접근해 헤엄쳐도 경계하지 않고 , 이빨 크기mm 3지 안팎이라고 합니다 .
고래상어는 어류 중 가장 덩치가 큽니다 . 하지만 상어 접근 이름에 걸맞지 않게 온순해서 사람이 가까이 안해 헤엄쳐도 경계하지 않고 , 이빨 크기도 3mm라는팎이라고 합니다 .


In [5]:
from ktextaug import random_insert, random_delete, back_translate
print(agent.generate(text,
                     mode="random_insert",
                     n_inserts=2))   

print(random_insert(text_or_tokens=text,
                     tokenizer=tokenizer, 
                     rng=Random(2021),
                     n_inserts=2))

고래상어는 어류 중 가장 덩치가 큽니다 . 하지만 상어라는 이름에 걸맞지 않게 온순해서 사람이 가까이 접근해 헤엄쳐도 size 경계 덩치하지 않고 , 이빨 크기도 3mm 안팎이라고 합니다 .
고래상어는 어류 중 가장 덩치가 큽니다 . 하지만 상어라는 높이 이름에 걸맞지 않게 온순해서 사람이 가까이 접근해 헤엄쳐도 경계하지 teeth 않고 , 이빨 크기도 3mm 안팎이라고 합니다 .


In [5]:
print(agent.generate(text,
                     mode="random_delete",
                     prob=0.1))   

print(random_delete(text_or_tokens=text,
                     tokenizer=tokenizer,
                     rng=Random(2021),
                     prob=0.1))

고래상어는 어류 중 가장 덩치가 큽니다 하지만 상어라는 이름에 걸맞지 온순해서 사람이 가까이 접근해 헤엄쳐도 경계하지 이빨 크기 3mm 안팎이라고 합니다
고래상어는 어류 중 가장 덩치가 큽니다 하지만 상어라는 걸맞지 않게 온순해서 사람이 가까이 접근해 헤엄쳐도 경계하지 않고 이빨 크기 3mm 안팎이라고 합니다


In [6]:
print(agent.generate(text,
                     mode="back_translate",
                     target_language='ja'))   

print(back_translate(text_or_corpus=text,
                     target_language='fr'))

고래 상어는 어류 중 가장 규모가 큽니다. 그러나 상어라는 이름에 걸맞게 않게 길들여 사람이 접근 수영 리노베이션 경계 않고 치아의 크기도 3mm 전후합니다.
고래 상어는 모든 물고기 중에서 가장 큽니다. 그러나 유순하고 상어의 이름에 걸맞지 않기 때문에 사람이 다가와 헤엄 쳐도 의심스럽지 않고 이빨이 3mm 정도라고한다.


### noise_generation 모듈 사용법

노이즈 생성은 @hkjeon13(전현규) 의 노이즈 생성을 따랐습니다

https://github.com/hkjeon13/noising-korean

노이즈를 생성하는 방법은 총 3가지가 구현되어 있습니다.

"jamo_split": 자모 분리(alphabet separation)에 의한 노이즈 추가 방법. 글자의 자음과 모음을 분리합니다. 단, 가독성을 위해 종성이 없으며 중성이  'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅗ' 가 아닐 경우 실행합니다(예: 안녕하세요 > 안녕ㅎㅏㅅㅔ요)

"vowel_change": 모음 변형에 의한 노이즈 추가 방법. 글자의 모음을 변형시킵니다. 단, 가독성을 위해 종성이 없으며 중성이 'ㅏ', 'ㅑ', 'ㅓ', 'ㅕ', 'ㅗ', 'ㅛ', 'ㅜ', 'ㅠ' 일 경우 실행합니다(예: 안녕하세요 > 안녕햐세오).

"phonological_change": 음운변화에 의한 노이즈 추가 방법. 발음을 바탕으로 단어를 변형시킵니다(너무 닮았다 > 너무 달맜다).

In [9]:
from ktextaug import TextAugmentation
from ktextaug import noise_add # function

text = '행복한 가정은 모두가 닮았지만, 불행한 가정은 모두 저마다의 이유로 불행하다.'
agent = TextAugmentation(tokenizer="mecab",
                        num_processes=1) 
print(agent.generate(text,
                     mode="noise_add",
                     prob=0.4,
                     noise_mode=['phonological_change', 'vowel_change', 'jamo_split']))   
# >> 행복한 ㄱㅏ정은 모두ㄱㅏ 닮았ㅈㅣ만, 불행한 ㄱㅏ정은 모두 ㅈㅓㅁㅏㄷㅏ의 ㅇㅣ유로 불행ㅎㅏㄷㅏ.

행복한 가정은 모두갸 닮았지만, 불행한 가정은 모듀 져먀다의 이우로 불행햐다.




**노이즈 생성의 변형 예시**

```python
text = '행복한 가정은 모두가 닮았지만, 불행한 가정은 모두 저마다의 이유로 불행하다.'
[original]  행복한 가정은 모두가 닮았지만, 불행한 가정은 모두 저마다의 이유로 불행하다.

[jamo_split, prob=1] 행복한 ㄱㅏ정은 모두ㄱㅏ 닮았ㅈㅣ만, 불행한 ㄱㅏ정은 모두 ㅈㅓㅁㅏㄷㅏ의 ㅇㅣ유로 불행ㅎㅏㄷㅏ.

[vowel_change, prob=1] 행복한 갸정은 묘듀갸 닮았지만, 불행한 갸정은 묘듀 져먀댜의 이우료 불행햐댜.

[phonological_change, prob=1] 행복한 가정은 모두가 달맜지만, 불행한 가정은 모두 저마다의 이유로 불행하다.
```

#### 기타

- 'phonological_change' 방법은 현재 비음화, 유음화, 구개음화, 연음 등을 구현하고 있으며, 추후 확대될 예정입니다(누락된 규칙이 있을 수 있으니, 발견 시 피드백 주시면 감사하겠습니다).
- prob는 변형 가능한 글자들에 대해서 해당 확률만큼 확률적으로 실행됩니다(prob가 1이라고 해서 모든 텍스트가 변경되는 것이 아닙니다).